In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
from sklearn.datasets import fetch_openml
data_mnist = fetch_openml('mnist_784', version=1)
data_mnist.target = data_mnist.target.astype(np.uint8)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data_mnist.data, data_mnist.target, test_size=0.1, random_state =1234)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state = 1234)

In [7]:
print("X_train: ", {X_train.shape})
print("X_val: ", {X_val.shape})
print("X_test: ", {X_test.shape})
print("y_train: ", {y_train.shape})
print("y_val: ", {y_val.shape})
print("y_test: ", {y_test.shape})

X_train:  {(56700, 784)}
X_val:  {(6300, 784)}
X_test:  {(7000, 784)}
y_train:  {(56700,)}
y_val:  {(6300,)}
y_test:  {(7000,)}


In [8]:
y_test.value_counts()

1    810
7    732
4    719
3    717
9    709
2    703
6    670
8    663
0    647
5    630
Name: class, dtype: int64

In [9]:
y_test

27504    6
57935    6
40387    7
24455    7
66362    4
        ..
35774    1
39638    4
7920     2
66773    2
41494    3
Name: class, Length: 7000, dtype: uint8

### Model

*   **relu** as activation function in hidden layers,
*   Use proper activation function in output layers.
*   Set proper input and output number of neurons.
*   Add **accuracy** metric
*   proper **loss** function,
*   use **adam** optimizer.

In [10]:
def create_and_compile_model():
  # define your model here:
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(256, activation="relu", input_shape=(28*28,)),
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(10, activation="softmax")
  ])
  model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"], optimizer="adam")
  return model

In [ ]:
# Compile

In [11]:
model = create_and_compile_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               200960    
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dense_4 (Dense)             (None, 10)                2570      
                                                                 
Total params: 400,906
Trainable params: 400,906
Non-trainable params: 0
_________________________________________________________________


In [12]:
predictions = model.predict(X_val/255.0)

197/197 [==============================] - 1s 3ms/step


In [ ]:
# Train with early stopping

In [13]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs = 50, validation_data=(X_val, y_val), batch_size = 64, callbacks=[es])

Epoch 1/50
886/886 [==============================] - 15s 15ms/step - loss: 0.7407 - accuracy: 0.8917 - val_loss: 0.2070 - val_accuracy: 0.9400
Epoch 2/50
886/886 [==============================] - 12s 13ms/step - loss: 0.1652 - accuracy: 0.9524 - val_loss: 0.1955 - val_accuracy: 0.9460
Epoch 3/50
886/886 [==============================] - 13s 14ms/step - loss: 0.1261 - accuracy: 0.9625 - val_loss: 0.1478 - val_accuracy: 0.9606
Epoch 4/50
886/886 [==============================] - 11s 12ms/step - loss: 0.1112 - accuracy: 0.9672 - val_loss: 0.1644 - val_accuracy: 0.9598
Epoch 5/50
886/886 [==============================] - 11s 12ms/step - loss: 0.1010 - accuracy: 0.9702 - val_loss: 0.1837 - val_accuracy: 0.9524
Epoch 6/50
886/886 [==============================] - 10s 12ms/step - loss: 0.0844 - accuracy: 0.9751 - val_loss: 0.1412 - val_accuracy: 0.9659
Epoch 7/50
886/886 [==============================] - 10s 12ms/step - loss: 0.0783 - accuracy: 0.9771 - val_loss: 0.1532 - val_accuracy:

In [ ]:
assert "accuracy" in history.history, "Use the accuracy metric!"
assert "val_loss" in history.history, "Use validation data!"
pd.DataFrame(history.history)[["loss", "val_loss"]].plot()
pd.DataFrame(history.history)[["accuracy", "val_accuracy"]].plot()

In [ ]:
# Test model

In [14]:
test_predictions = model.evaluate(X_test, y_test)

219/219 [==============================] - 1s 4ms/step - loss: 0.1258 - accuracy: 0.9734


Test on random value from test set

In [15]:
import cv2
from google.colab.patches import cv2_imshow

image = X_test.sample()

cv2_imshow(image.to_numpy().reshape(28,28,1))
prediction = model.predict(image)
print("Predicted class", prediction.argmax(axis=1))

1/1 [==============================] - 0s 39ms/step
Predicted class [5]


In [16]:
prediction[0]

array([7.0045111e-26, 3.5500654e-26, 1.7786329e-35, 6.5125162e-15,
       2.8814135e-30, 1.0000000e+00, 2.6735083e-21, 0.0000000e+00,
       8.7904451e-20, 1.2437408e-24], dtype=float32)